# Библиотеки

In [2]:
import pandas as pd
import psycopg2

# Актуализация БД

In [8]:
# Функция подключения к базе данных
def connect_to_db():
    try:
        conn = psycopg2.connect(
            dbname="postgres",
            user="postgres",
            password="1234",
            host="localhost",
            port="5432"
        )
        return conn
    except Exception as e:
        print(f"Ошибка подключения к базе данных: {e}")
        return None

# Чтение CSV-файла
def read_csv(file_path):
    return pd.read_csv(file_path)

# Обновление или добавление статьи
def upsert_article(conn, article_id, title, in_rinc):
    cursor = conn.cursor()
    try:
        # Проверка наличия статьи
        cursor.execute("SELECT id FROM elibrary.Articles WHERE id = %s", (article_id,))
        existing_article = cursor.fetchone()

        if existing_article:
            # Обновить поле in_rinc, если оно изменилось
            cursor.execute("""
                UPDATE elibrary.Articles
                SET in_rinc = %s
                WHERE id = %s
            """, (in_rinc, article_id))
        else:
            # Добавить новую статью
            cursor.execute("""
                INSERT INTO elibrary.Articles (id, title, in_rinc)
                VALUES (%s, %s, %s)
            """, (article_id, title, in_rinc))
        conn.commit()
    except Exception as e:
        print(f"Ошибка при обновлении/добавлении статьи: {e}")
        conn.rollback()

# Добавление авторов
def add_authors(conn, article_id, authors_str):
    cursor = conn.cursor()
    try:
        if authors_str.strip().lower() == "нет авторов":
            return  # Пропускаем, если нет авторов

        authors = [author.strip() for author in authors_str.split(",")]

        for author in authors:
            # Проверяем, есть ли уже такой автор для этой статьи
            cursor.execute("""
                SELECT id FROM elibrary.authors
                WHERE article_id = %s AND author_name = %s
            """, (article_id, author))
            if cursor.fetchone():
                continue  # Автор уже есть, пропускаем

            cursor.execute("""
                INSERT INTO elibrary.authors (article_id, author_name, contribution, applied_for_award)
                VALUES (%s, %s, %s, %s)
            """, (article_id, author, 0, False))
        conn.commit()
    except Exception as e:
        print(f"Ошибка при добавлении авторов: {e}")
        conn.rollback()

# Основная функция
def main(csv_file_path):
    conn = connect_to_db()
    if not conn:
        return

    try:
        df = read_csv(csv_file_path)

        for _, row in df.iterrows():
            article_id = int(row["ID"])
            title = row["Название"]
            in_rinc = row["В_РИНЦ"].strip().lower() == "да"
            authors_str = row["Авторы"]

            # Обновление или добавление статьи
            upsert_article(conn, article_id, title, in_rinc)

            # Добавление авторов
            add_authors(conn, article_id, authors_str)

        print("Данные успешно добавлены в базу данных!")

    except Exception as e:
        print(f"Ошибка: {e}")
    finally:
        conn.close() 

# Запуск программы
if __name__ == "__main__":
    csv_file_path = "final_result.csv"  # Укажите путь к вашему CSV-файлу
    main(csv_file_path)

Данные успешно добавлены в базу данных!
